In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms
from torchvision import models
from torchvision.models import vgg16,resnet50, MobileNetV3, Inception_V3_Weights, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from collections import Counter

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Class names
class_names = [
    'animal fish',
    'animal fish bass',
    'fish sea_food black_sea_sprat',
    'fish sea_food gilt_head_bream',
    'fish sea_food hourse_mackerel',
    'fish sea_food red_mullet',
    'fish sea_food red_sea_bream',
    'fish sea_food sea_bass',
    'fish sea_food shrimp',
    'fish sea_food striped_red_mullet',
    'fish sea_food trout'
]

# Paths
img_path_train = r"C:\Users\achu1\Documents\GUVI\Project -5\data\train"
img_path_val = r"C:\Users\achu1\Documents\GUVI\Project -5\data\val"
img_path_test = r"C:\Users\achu1\Documents\GUVI\Project -5\data\test"

# Transforms
trans = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Load datasets
train_data = datasets.ImageFolder(root=img_path_train, transform=trans)
val_data = datasets.ImageFolder(root=img_path_val, transform=trans)
test_data = datasets.ImageFolder(root=img_path_test, transform=trans)

# DataLoaders
train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=32)
test_data_loader = DataLoader(test_data, batch_size=32)

# Print class distribution
print("Class Distribution:", Counter(train_data.targets))


Class Distribution: Counter({0: 1096, 10: 580, 5: 579, 8: 576, 4: 573, 6: 571, 2: 569, 3: 566, 9: 547, 7: 538, 1: 30})


In [2]:
import pickle

class_names = [
    'animal fish',
    'animal fish bass',
    'fish sea_food black_sea_sprat',
    'fish sea_food gilt_head_bream',
    'fish sea_food hourse_mackerel',
    'fish sea_food red_mullet',
    'fish sea_food red_sea_bream',
    'fish sea_food sea_bass',
    'fish sea_food shrimp',
    'fish sea_food striped_red_mullet',
    'fish sea_food trout'
]

with open("class_names.pkl", "wb") as f:
    pickle.dump(class_names, f)


In [3]:
# Model
model = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(16, 32, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(32, 64, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Flatten(),
    nn.Linear(64 * 16 * 16, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, len(class_names))
).to(device)

# Loss and optimizer
model_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [4]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = model_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {running_loss:.4f}")
    

Epoch 1/50 - Training Loss: 254.6331
Epoch 2/50 - Training Loss: 114.7357
Epoch 3/50 - Training Loss: 71.8091
Epoch 4/50 - Training Loss: 58.5423
Epoch 5/50 - Training Loss: 44.5056
Epoch 6/50 - Training Loss: 34.8796
Epoch 7/50 - Training Loss: 34.4719
Epoch 8/50 - Training Loss: 29.0318
Epoch 9/50 - Training Loss: 26.1810
Epoch 10/50 - Training Loss: 18.1689
Epoch 11/50 - Training Loss: 19.3811
Epoch 12/50 - Training Loss: 19.4668
Epoch 13/50 - Training Loss: 15.6842
Epoch 14/50 - Training Loss: 15.4212
Epoch 15/50 - Training Loss: 17.5364
Epoch 16/50 - Training Loss: 13.9419
Epoch 17/50 - Training Loss: 15.7775
Epoch 18/50 - Training Loss: 12.2194
Epoch 19/50 - Training Loss: 10.0567
Epoch 20/50 - Training Loss: 12.9912
Epoch 21/50 - Training Loss: 11.4044
Epoch 22/50 - Training Loss: 9.6504
Epoch 23/50 - Training Loss: 12.1612
Epoch 24/50 - Training Loss: 14.0002
Epoch 25/50 - Training Loss: 10.8381
Epoch 26/50 - Training Loss: 7.4538
Epoch 27/50 - Training Loss: 13.4696
Epoch 28/5

In [5]:
  # Validate
model.eval()
correct, total = 0, 0
with torch.no_grad():
        for images, labels in val_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")

# Function to predict class of an image
def pred_img_class(img_path):
    model.eval()
    image = Image.open(img_path).convert("RGB")
    image_tensor = trans(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        _, pred = torch.max(output, 1)
        predicted_class = class_names[pred.item()]
        print("Predicted Class:", predicted_class)
        return predicted_class

Validation Accuracy: 98.53%


In [28]:
# Final accuracy calculation
def calculate_accuracy(model, data_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    A1 = 100 * correct / total
    print(f"Accuracy: {A1:.2f}%")
    return A1 #Accuracy

# Try predicting an image
pred_img_class(r"C:\Users\achu1\Downloads\AIML Tamil Recordings\Project orientation\Project 5-Multiclass Fish Image Classification\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\train\fish sea_food trout\1RNWEVD9TZH3.jpg")

# Final accuracy on train/val/test
calculate_accuracy(model, train_data_loader)
calculate_accuracy(model, val_data_loader)
calculate_accuracy(model, test_data_loader)

A1 = calculate_accuracy(model, test_data_loader)
print("Test Accuracy:", A1)

Predicted Class: fish sea_food trout
Accuracy: 99.92%
Accuracy: 98.99%
Accuracy: 99.40%
Accuracy: 99.47%
Test Accuracy: 99.46658299341073


In [29]:
#torch.save(model, "fish_classifier_model.pth") 
torch.save(model, "cnn_model.pth") 

In [30]:
#EX
pred_img_class(r"C:\Users\achu1\Downloads\AIML Tamil Recordings\Project orientation\Project 5-Multiclass Fish Image Classification\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\train\fish sea_food hourse_mackerel\GEV2Q44R5ENG.jpg")

# Final accuracy on train/val/test
calculate_accuracy(model, train_data_loader)
calculate_accuracy(model, val_data_loader)
calculate_accuracy(model, test_data_loader)

Predicted Class: fish sea_food hourse_mackerel
Accuracy: 99.95%
Accuracy: 98.53%
Accuracy: 99.50%


99.49796046438657

In [31]:
pred_img_class(r"C:\Users\achu1\Downloads\AIML Tamil Recordings\Project orientation\Project 5-Multiclass Fish Image Classification\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\test\fish sea_food shrimp\6ZYXHRU8APXV.jpg")

# Final accuracy on train/val/test
calculate_accuracy(model, train_data_loader)
calculate_accuracy(model, val_data_loader)
calculate_accuracy(model, test_data_loader)

Predicted Class: fish sea_food shrimp
Accuracy: 99.94%
Accuracy: 98.63%
Accuracy: 99.50%


99.49796046438657

## VGG16:

In [33]:
# Model
vgg = vgg16(pretrained=True)
for param in vgg.features.parameters():
    param.requires_grad = False

# Replace the classifier
vgg.classifier = nn.Sequential(
    nn.Linear(25088, 512),  # 25088 = 512 * 7 * 7 (flattened VGG16 output)
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(class_names))  # your number of classes
)

vgg = vgg.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg.classifier.parameters(), lr=0.001) 

C:\Users\achu1\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\achu1\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    vgg.train()
    running_loss = 0.0
    for images, labels in train_data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg(images)  # not vgg.classifier(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {running_loss:.4f}")



Epoch 1/20 - Training Loss: 109.6468
Epoch 2/20 - Training Loss: 40.3600
Epoch 3/20 - Training Loss: 36.9347
Epoch 4/20 - Training Loss: 36.2121
Epoch 5/20 - Training Loss: 28.0188
Epoch 6/20 - Training Loss: 24.9327
Epoch 7/20 - Training Loss: 18.4582
Epoch 8/20 - Training Loss: 23.3316
Epoch 9/20 - Training Loss: 25.6504
Epoch 10/20 - Training Loss: 28.1144
Epoch 11/20 - Training Loss: 20.5447
Epoch 12/20 - Training Loss: 22.2787
Epoch 13/20 - Training Loss: 23.0533
Epoch 14/20 - Training Loss: 21.4129
Epoch 15/20 - Training Loss: 25.2599
Epoch 16/20 - Training Loss: 20.8212
Epoch 17/20 - Training Loss: 21.9612
Epoch 18/20 - Training Loss: 17.2629
Epoch 19/20 - Training Loss: 31.1507
Epoch 20/20 - Training Loss: 30.5610


In [35]:
  # Validate
vgg.eval()
correct, total = 0, 0
with torch.no_grad():
        for images, labels in val_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")

# Function to predict class of an image
def pred_img_class(img_path):
    vgg.eval()
    image = Image.open(img_path).convert("RGB")
    image_tensor = trans(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = vgg(image_tensor)
        _, pred = torch.max(output, 1)
        predicted_class = class_names[pred.item()]
        print("Predicted Class:", predicted_class)
        return predicted_class

Validation Accuracy: 97.99%


In [36]:
# Final accuracy calculation
def calculate_accuracy(model, data_loader):
    vgg.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in train_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    A2 = 100 * correct / total
    print(f"Accuracy: {A2:.2f}%")
    return A2

# Try predicting an image
pred_img_class(r"C:\Users\achu1\Downloads\AIML Tamil Recordings\Project orientation\Project 5-Multiclass Fish Image Classification\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\train\fish sea_food trout\1RNWEVD9TZH3.jpg")

# Final accuracy on train/val/test
calculate_accuracy(vgg, train_data_loader)
calculate_accuracy(vgg, val_data_loader)
calculate_accuracy(vgg, test_data_loader)

A2 = calculate_accuracy(vgg, test_data_loader)
print("Test Accuracy:", A2)

Predicted Class: fish sea_food trout
Accuracy: 98.94%
Accuracy: 98.99%
Accuracy: 99.02%
Accuracy: 98.97%
Test Accuracy: 98.9718875502008


In [37]:
torch.save(vgg, "vgg16_model.pth") 

## Resnet50

In [39]:
# Load ResNet50
resnet = resnet50(pretrained=True)

# Freeze feature extractor layers
for param in resnet.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(class_names))
)

resnet = resnet.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

C:\Users\achu1\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0

    for images, labels in train_data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {running_loss:.4f}")




Epoch 1/20 - Training Loss: 124.0024
Epoch 2/20 - Training Loss: 60.4665
Epoch 3/20 - Training Loss: 55.7706
Epoch 4/20 - Training Loss: 45.9367
Epoch 5/20 - Training Loss: 38.5934
Epoch 6/20 - Training Loss: 39.1693
Epoch 7/20 - Training Loss: 41.1560
Epoch 8/20 - Training Loss: 36.9266
Epoch 9/20 - Training Loss: 36.6242
Epoch 10/20 - Training Loss: 36.4609
Epoch 11/20 - Training Loss: 33.2191
Epoch 12/20 - Training Loss: 31.2902
Epoch 13/20 - Training Loss: 31.8092
Epoch 14/20 - Training Loss: 31.0786
Epoch 15/20 - Training Loss: 30.4065
Epoch 16/20 - Training Loss: 27.1667
Epoch 17/20 - Training Loss: 32.6478
Epoch 18/20 - Training Loss: 27.5814
Epoch 19/20 - Training Loss: 27.2148
Epoch 20/20 - Training Loss: 30.1436


In [41]:
# Validation
resnet.eval()
correct, total = 0, 0
with torch.no_grad():
        for images, labels in train_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = resnet(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")


Validation Accuracy: 98.57%


In [42]:

# Prediction function
def pred_img_class(img_path):
    resnet.eval()
    image = Image.open(img_path).convert("RGB")
    image_tensor = trans(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = resnet(image_tensor)
        _, pred = torch.max(output, 1)
        predicted_class = class_names[pred.item()]
        print("Predicted Class:", predicted_class)
        return predicted_class

# Accuracy function
def calculate_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    A3 = 100 * correct / total
    print(f"Accuracy: {A3:.2f}%")
    return A3


# Example usage
pred_img_class(r"C:\Users\achu1\Downloads\AIML Tamil Recordings\Project orientation\Project 5-Multiclass Fish Image Classification\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\train\fish sea_food trout\1RNWEVD9TZH3.jpg")

calculate_accuracy(resnet, train_data_loader)
calculate_accuracy(resnet, val_data_loader)
calculate_accuracy(resnet, test_data_loader)

A3 = calculate_accuracy(resnet, test_data_loader)
print("Test Accuracy:", A3)

Predicted Class: fish sea_food trout
Accuracy: 98.59%
Accuracy: 97.44%
Accuracy: 97.30%
Accuracy: 97.14%
Test Accuracy: 97.14465014119862


In [43]:
torch.save(resnet, "resnet_model.pth")

## Mobilenet

In [45]:
# Load MobileNetV3 (large version)
mobilenet = models.mobilenet_v3_large(pretrained=True)

# Freeze feature extractor
for param in mobilenet.features.parameters():
    param.requires_grad = False

# Replace the classifier
mobilenet.classifier = nn.Sequential(
    nn.Linear(mobilenet.classifier[0].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(class_names))
)

mobilenet = mobilenet.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.classifier.parameters(), lr=0.001)





C:\Users\achu1\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [46]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    mobilenet.train()
    running_loss = 0.0

    for images, labels in train_data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = mobilenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {running_loss:.4f}")




Epoch 1/20 - Training Loss: 102.5968
Epoch 2/20 - Training Loss: 44.8658
Epoch 3/20 - Training Loss: 38.3672
Epoch 4/20 - Training Loss: 34.1510
Epoch 5/20 - Training Loss: 28.9037
Epoch 6/20 - Training Loss: 24.7364
Epoch 7/20 - Training Loss: 22.1588
Epoch 8/20 - Training Loss: 20.9949
Epoch 9/20 - Training Loss: 21.8207
Epoch 10/20 - Training Loss: 19.3542
Epoch 11/20 - Training Loss: 20.2246
Epoch 12/20 - Training Loss: 17.9877
Epoch 13/20 - Training Loss: 17.3941
Epoch 14/20 - Training Loss: 16.8036
Epoch 15/20 - Training Loss: 17.0379
Epoch 16/20 - Training Loss: 16.6660
Epoch 17/20 - Training Loss: 16.3851
Epoch 18/20 - Training Loss: 15.0288
Epoch 19/20 - Training Loss: 15.3170
Epoch 20/20 - Training Loss: 13.7174


In [47]:
    # Validation
    mobilenet.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = mobilenet(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

Validation Accuracy: 97.34%


In [48]:


# Prediction function
def pred_img_class(img_path):
    mobilenet.eval()
    image = Image.open(img_path).convert("RGB")
    image_tensor = trans(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = mobilenet(image_tensor)
        _, pred = torch.max(output, 1)
        predicted_class = class_names[pred.item()]
        print("Predicted Class:", predicted_class)
        return predicted_class


# Accuracy function
def calculate_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    A4 = 100 * correct / total
    print(f"Accuracy: {A4:.2f}%")
    return A4


# Test it
pred_img_class(r"C:\Users\achu1\Downloads\AIML Tamil Recordings\Project orientation\Project 5-Multiclass Fish Image Classification\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\train\fish sea_food trout\1RNWEVD9TZH3.jpg")

calculate_accuracy(mobilenet, train_data_loader)
calculate_accuracy(mobilenet, val_data_loader)
calculate_accuracy(mobilenet, test_data_loader)

A4 = calculate_accuracy(mobilenet, test_data_loader)
print("Test Accuracy:", A4)

Predicted Class: fish sea_food trout
Accuracy: 98.34%
Accuracy: 98.46%
Accuracy: 98.43%
Accuracy: 98.31%
Test Accuracy: 98.30561656730468


In [49]:
torch.save(mobilenet, "mobilenet.pth") 

## InceptionV3:

In [51]:
# Transforms
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # <- Resize to 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_data = datasets.ImageFolder(root=img_path_train, transform=transform)
val_data = datasets.ImageFolder(root=img_path_val, transform=transform)
test_data = datasets.ImageFolder(root=img_path_test, transform=transform)

# DataLoaders
train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=32)
test_data_loader = DataLoader(test_data, batch_size=32)

# Load Inception V3 (pretrained)
inception = models.inception_v3(pretrained=True)

# Freeze all layers except the final classifier
for param in inception.parameters():
    param.requires_grad = False

# Replace the classifier for our task
inception.AuxLogits.fc = nn.Sequential(
    nn.Linear(inception.AuxLogits.fc.in_features, len(class_names))
)

inception.fc = nn.Sequential(
    nn.Linear(inception.fc.in_features, len(class_names))
)

# Move model to device (GPU or CPU)
inception = inception.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(inception.fc.parameters(), lr=0.001)


C:\Users\achu1\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [52]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    inception.train()
    running_loss = 0.0

    for images, labels in train_data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs,aux_outputs = inception(images)
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2 
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {running_loss:.4f}")

Epoch 1/20 - Training Loss: 376.2054
Epoch 2/20 - Training Loss: 261.5044
Epoch 3/20 - Training Loss: 239.1089
Epoch 4/20 - Training Loss: 229.9713
Epoch 5/20 - Training Loss: 225.0884
Epoch 6/20 - Training Loss: 221.4644
Epoch 7/20 - Training Loss: 219.4487
Epoch 8/20 - Training Loss: 217.8293
Epoch 9/20 - Training Loss: 214.2628
Epoch 10/20 - Training Loss: 214.9523
Epoch 11/20 - Training Loss: 213.3569
Epoch 12/20 - Training Loss: 210.6734
Epoch 13/20 - Training Loss: 212.2096
Epoch 14/20 - Training Loss: 210.5636
Epoch 15/20 - Training Loss: 210.8212
Epoch 16/20 - Training Loss: 208.7155
Epoch 17/20 - Training Loss: 208.3593
Epoch 18/20 - Training Loss: 207.7360
Epoch 19/20 - Training Loss: 208.7554
Epoch 20/20 - Training Loss: 208.4075


In [53]:
    # Validation
inception.eval()
correct, total = 0, 0
with torch.no_grad():
        for images, labels in val_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = inception(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

Validation Accuracy: 98.81%


In [54]:


# Prediction function
def pred_img_class(img_path):
    inception.eval()
    image = Image.open(img_path).convert("RGB")
    image_tensor = trans(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = inception(image_tensor)
        _, pred = torch.max(output, 1)
        predicted_class = class_names[pred.item()]
        print("Predicted Class:", predicted_class)
        return predicted_class


# Accuracy function
def calculate_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    A5 = 100 * correct / total
    print(f"Accuracy: {A5:.2f}%")
    return A5


# Test the model with an image
pred_img_class(r"C:\Users\achu1\Downloads\AIML Tamil Recordings\Project orientation\Project 5-Multiclass Fish Image Classification\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\train\fish sea_food trout\1RNWEVD9TZH3.jpg")

# Final accuracy on train/val/test
calculate_accuracy(inception, train_data_loader)
calculate_accuracy(inception, val_data_loader)
calculate_accuracy(inception, test_data_loader)

A5 = calculate_accuracy(inception, test_data_loader)
print("Test Accuracy:", A5)

Predicted Class: fish sea_food trout
Accuracy: 99.22%
Accuracy: 99.22%
Accuracy: 99.22%
Accuracy: 99.22%
Test Accuracy: 99.21556322560401


In [55]:
torch.save(inception, "inception.pth")

## EfficientNet0

In [57]:
#pip install efficientnet_pytorch

In [58]:
from efficientnet_pytorch import EfficientNet

# Transforms
trans = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Load datasets
train_data = datasets.ImageFolder(root=img_path_train, transform=trans)
val_data = datasets.ImageFolder(root=img_path_val, transform=trans)
test_data = datasets.ImageFolder(root=img_path_test, transform=trans)

# DataLoaders
train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=32)
test_data_loader = DataLoader(test_data, batch_size=32)

# Load EfficientNetB0 (pretrained)
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')

# Freeze all layers except the final classifier
for param in efficientnet.parameters():
    param.requires_grad = False

# Replace the classifier for our task
efficientnet._fc = nn.Sequential(
    nn.Linear(efficientnet._fc.in_features, len(class_names))
)

# Move model to device (GPU or CPU)
efficientnet = efficientnet.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet._fc.parameters(), lr=0.001)




Loaded pretrained weights for efficientnet-b0


In [59]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    efficientnet.train()
    running_loss = 0.0

    for images, labels in train_data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = efficientnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {running_loss:.4f}")



Epoch 1/20 - Training Loss: 183.8495
Epoch 2/20 - Training Loss: 80.2207
Epoch 3/20 - Training Loss: 62.4017
Epoch 4/20 - Training Loss: 52.3834
Epoch 5/20 - Training Loss: 44.3826
Epoch 6/20 - Training Loss: 40.7085
Epoch 7/20 - Training Loss: 38.9286
Epoch 8/20 - Training Loss: 35.9660
Epoch 9/20 - Training Loss: 34.0754
Epoch 10/20 - Training Loss: 34.7642
Epoch 11/20 - Training Loss: 32.3003
Epoch 12/20 - Training Loss: 31.7812
Epoch 13/20 - Training Loss: 29.3900
Epoch 14/20 - Training Loss: 28.8326
Epoch 15/20 - Training Loss: 27.3397
Epoch 16/20 - Training Loss: 26.2493
Epoch 17/20 - Training Loss: 26.7187
Epoch 18/20 - Training Loss: 26.9549
Epoch 19/20 - Training Loss: 25.1639
Epoch 20/20 - Training Loss: 26.2793


In [60]:

    # Validation
    efficientnet.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = efficientnet(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

Validation Accuracy: 97.53%


In [61]:



# Prediction function
def pred_img_class(img_path):
    efficientnet.eval()
    image = Image.open(img_path).convert("RGB")
    image_tensor = trans(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = efficientnet(image_tensor)
        _, pred = torch.max(output, 1)
        predicted_class = class_names[pred.item()]
        print("Predicted Class:", predicted_class)
        return predicted_class

# Accuracy function
def calculate_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    A6 = 100 * correct / total
    print(f"Accuracy: {A6:.3f}%")
    return A6

# Test the model with an image
pred_img_class(r"C:\Users\achu1\Downloads\AIML Tamil Recordings\Project orientation\Project 5-Multiclass Fish Image Classification\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\train\fish sea_food trout\1RNWEVD9TZH3.jpg")

# Final accuracy on train/val/test
calculate_accuracy(efficientnet, train_data_loader)
calculate_accuracy(efficientnet, val_data_loader)
calculate_accuracy(efficientnet, test_data_loader)
A6 = calculate_accuracy(efficientnet, test_data_loader)
print("Test Accuracy:", A6)


Predicted Class: fish sea_food trout
Accuracy: 98.274%
Accuracy: 98.117%
Accuracy: 98.023%
Accuracy: 98.055%
Test Accuracy: 98.05459679949796


In [62]:
torch.save(efficientnet, "efficient.pth") 

In [63]:
A5

99.21556322560401

## Saving Model:

In [65]:

model_accuracies = {
    "CNN": A1,
    "VGG16": A2,
    "ResNet50": A3,
    "MobileNetV2": A4,
    "InceptionV3": A5,
    "EfficientNetB0": A6,
}

# Find best model name
best_model_name = max(model_accuracies, key=model_accuracies.get)
print(f"✅ Best Model: {best_model_name} with Accuracy: {model_accuracies[best_model_name]:.2f}%")

# Now load the best model (if models already saved separately)
model_path_map = {
    "CNN": "cnn_model.pth",
    "VGG16": "vgg16_model.pth",
    "ResNet50": "resnet_model.pth",
    "MobileNetV2": "mobilenet.pth",
    "InceptionV3": "inception.pth",
    "EfficientNetB0": "efficient.pth",
}

# Load the actual model file
best_model = torch.load(model_path_map[best_model_name],map_location=device, weights_only=False)

# Save best model under new name
torch.save(best_model, "best_fish_model.pth")
print("✅ Saved the best model as best_fish_model.pth")



✅ Best Model: CNN with Accuracy: 99.47%
✅ Saved the best model as best_fish_model.pth
